## Tweedie regression on insurance claims

This example illustrates the use of Poisson, Gamma and Tweedie regression on the [French Motor Third-Party Liability Claims dataset](https://www.openml.org/d/41214), and is inspired by an R tutorial.

这个例子说明了在法国汽车第三方责任索赔数据集上使用泊松、伽马和特威迪回归，并且受到了一个 R 教程的启发。

In this dataset, each sample corresponds to an insurance policy, i.e. a contract within an insurance company and an individual (policyholder). Available features include driver age, vehicle age, vehicle power, etc.

在这个数据集中，每个样本对应一份保险单，即保险公司内的一份合同和一个个人（投保人）。可用的特征包括驾驶员年龄、车辆使用年限、车辆动力等。

有效的特征：
| 特征名 | 说明 |
|:------|:----|
|driver age | 驾驶员年龄 |
|vehicle age | 车辆使用年限 |
|vehicle power | 车辆动力 |

A few definitions: a claim is the request made by a policyholder to the insurer to compensate for a loss covered by the insurance. The claim amount is the amount of money that the insurer must pay. The exposure is the duration of the insurance coverage of a given policy, in years.

一些有用的定义：

| 名词 | 说明 |
|:----|:----|
|claim | 索赔 |
|claim amount | 索赔金额 |
|exposure | 风险敞口，是给定保单的保险期限（以年为单位）|

Here our goal is to predict the expected value, i.e. the mean, of the total claim amount per exposure unit also referred to as the pure premium.

在这里，我们的目标是预测预期值，即每次风险暴露单位的总索赔金额的平均值，也称为纯保费。

There are several possibilities to do that, two of which are:
有几种实现的方式，其中两种是：

1. Model the number of claims with a Poisson distribution, and the average claim amount per claim, also known as severity, as a Gamma distribution and multiply the predictions of both in order to get the total claim amount.

用泊松分布对索赔数量进行建模，将每次索赔的平均索赔金额（也称为严重程度）用伽马分布建模，然后将两者的预测结果相乘，以得到总索赔金额。

| 特征名 | 说明 | 分布 | 建模方式 |
|:------|:----|:--------|:----|
| ClaimNb | 索赔数量 | 泊松分布(Poisson) | 泊松回归 |
| ClaimAmount | 每次索赔的平均索赔金额 | 伽马分布(Gamma) | 伽马回归 |

2. Model the total claim amount per exposure directly, typically with a Tweedie distribution of Tweedie power p ∈ (1, 2)

直接对每个风险敞口内的总索赔金额进行建模，通常使用特威迪幂的特威迪分布。

| 特征名 | 说明 | 分布 | 建模方式 |
|:------|:----|:----|:--------|
|ClaimAmount | 每个风险敞口内的总索赔金额 | 特威迪分布(Tweedie) | 特威迪回归 |


In this example we will illustrate both approaches. We start by defining a few helper functions for loading the data and visualizing results.

在这个例子中，我们将说明这两种方法。我们首先定义一些用于加载数据和可视化结果的辅助函数。

**相关文档：**
1. A. Noll, R. Salzmann and M.V. Wuthrich, Case Study: French Motor Third-Party Liability Claims (November 8, 2018). doi:10.2139/ssrn.3164764


## 原始特征
| 特征名 | 说明 | 特征预处理方法 | 特征类别 |
|:------|:----|:-------------|:-----------|
| ClaimNb | 风险敞口内的索赔次数 | - | **Y标签** |
| Exposure | 风险敞口，是给定保单的保险期限（以年为单位）| - | **Y标签** |
| ClaimAmount | 索赔金额 | - | **Y特征** |
| Area | 区域编码 | One-Hot | X特征 |
| VehPower | 车辆动力 | One-Hot | X特征 |
| VehAge | 车辆年龄 | 10分桶 | X特征 |
| DrivAge | 驾驶员年龄 | 10分桶 | X特征 |
| BonusMalus | 奖惩制度 奖励 / 惩罚，在 50 到 350 之间：在法国，小于 100 意味着奖励，大于 100 意味着惩罚。 | 无需处理 | X特征 |
| VehBrand | 车辆品牌 | One-Hot | X特征 |
| VehGas | 车辆油耗 | One-Hot | X特征 |
| Density | 汽车驾驶员所居住城市的居民密度（每平方公里的居民数量）。| 0-1标准化 | X特征 |
| Region | 地区 法国的保单地区（基于标准的法国分类）| One-Hot | X特征 |

## 衍生特征

This can be indirectly approximated by a 2-step modeling: the product of the

Frequency times the average claim amount per claim:

使用两步建模法时，需要求解的衍生特征

| 特征名 | 定义 | 说明 |
|:------|:----|:-----|
| PurePremium | df["PurePremium"] = df["ClaimAmount"] / df["Exposure"] | 纯保费, 保险公司对纯保费的建模很感兴趣，即其投资组合中每个投保人每单位风险敞口的预期总索赔金额： |
| Frequency | df["ClaimNb"] / df["Exposure"] | 索赔频率 |
| AvgClaimAmount | df["AvgClaimAmount"] = df["ClaimAmount"] / np.fmax(df["ClaimNb"], 1) | 每次索赔的平均索赔金额 |
